In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

ModuleNotFoundError: No module named 'google'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

Import the python libraries necessary for creating a Naive Bayes supervised machine learning algorithm used to classify email spam.

In [3]:
import os
import numpy as np
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

Create an "extract features" function to clean and prepare the data for the model. This entails removing the non required words, expressions and symbols from the text. The end result will be a matrix representation of the word frequency across each document, each item in the dictionary represented as a column and each email represented by a row.

In [4]:
# define the function that will extract features and create the dictionary
def make_Dictionary(root_dir):
# create an empty list
  all_words = []
# create the emails master list by combining the email paths into one single path
  emails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)]
# initiate a for loop to loop through each email
  for mail in emails:
# assign m to each email
    with open(mail) as m:
# initiate another for loop to loop through each line in m (the email), then split the words in the line, then add all the strings in words to all_words
      for line in m:
        words = line.split()
        all_words += words
# define the dictionary and fill it up by counting the number of occurences of each item (word) in all_words
  dictionary = Counter(all_words)
# create a list of iterables within dictionary (words) and assign the list to list_to_remove
  list_to_remove = list(dictionary)
#initiate a for loop for every item (words) in list_to_remove
  for item in list_to_remove:
# isalpha() returns “True” if all characters in the string are alphabetical (in [a-z or A-Z]) - else (if there are numbers or special characters), it returns “False”
    if item.isalpha() == False:
# if the string contains anything other than letters, delete the item from the dictionary
      del dictionary[item]
# else, if the string only is one letter long, delete the item from the dictionary
    elif len(item) == 1:
      del dictionary[item]
# keep only the most common 3,000 items in the dictionary
  dictionary = dictionary.most_common(3000)
# return the dictionary as the result of the first for loop
  return dictionary

This function will create feature columns and populate their value for each email. The Feature Matrix will have 3000 columns, representing the 3000 most common words we kept in the dictionary, and either 702 (training) or 260 (test) rows representing the number of email files.
The function also analyzes the name of each email file and decides if it's spam or not based on whether the filepath contains 'spmsg'. The function also creates a new column, Labelled Data Column, that labels the email as spam or not. 
This function is used to gather the training dataset, gather the emails for the testing dataset, and returns the feature dataset with the New Label column.

In [5]:
# define the function that will create the docterm matrix from the mail_dir 
def extract_features(mail_dir):
# use a for loop to join together the files using os.listdir to grab all files and directories in mail_dir and os.path.join to concatenate these path components
  files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
# create an array, filled with zeros in every place, with 3000 columns (for the 3000 most common words) and how many ever rows are in the mail_dir for the selected data
  features_matrix = np.zeros((len(files),3000))
# create a train labels array filled with zeros, the array will be one row with columns the length of the amount of files in the selected data
  train_labels = np.zeros(len(files))
# set the count equal to 1 to keep track of how many times the algorithm has iterated
  count = 1;
  docID = 0;
# initiate another for loop that will eventually tokenize the 
  for fil in files:
# open fil and return it as a file object
    with open(fil) as fi:
# for each element in fi, a tuple is produced with count, docID; the for loop binds that to i, line
      for i, line in enumerate(fi):
# if i exactly equals 2, then
        if i == 2:
# split line into a list
          words = line.split()
# initiate a for loop to iterate through each word in words
          for word in words:
# set wordID to zero
            wordID = 0
# initiate a for loop that will count the amount of times the word appears
            for i, d in enumerate(dictionary):
              if d[0] == word:
                wordID = i
                features_matrix[docID,wordID] = words.count(word)
# train the program to recognize whether or not the email is a spam email
      train_labels[docID] = 0;
#separate the first part of the file name before the '/'
      filepathTokens = fil.split('/')
      lastToken = filepathTokens[len(filepathTokens)-1]
# if the first part starts with spmsg
      if lastToken.startswith("spmsg"):
# the train column of this particular docID is 1, meaning yes it's spam
        train_labels[docID] = 1;
# keep track of how many times the loop has iterated
        count = count + 1
# keep track of the document ID
      docID = docID + 1
# when the function is called, return the matrix with the train column
  return features_matrix, train_labels

Call the two functions created above. First, we train the model using the model.fit function and Training Dataset. Then it evaluates the accuracy of the trained model by comparing it to the test data.

In [6]:
TRAIN_DIR = '/content/drive/My Drive/BSAN6070/Data/train-mails'
TEST_DIR = '/content/drive/My Drive/BSAN6070/Data/test-mails'

dictionary = make_Dictionary(TRAIN_DIR)

print ("Reading and processing emails from TRAIN and TEST folders")
features_matrix, labels = extract_features(TRAIN_DIR)
test_features_matrix, test_labels = extract_features(TEST_DIR)

model = GaussianNB()

print ("Training Model using Gaussian Naibe Bayes algorithm")
model.fit(features_matrix, labels)
print ("Training completed")
print ("Testing trained model to predict Test Data labels")
predicted_labels = model.predict(test_features_matrix)
print ("Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:")
print ("The accuracy score of this model is ", accuracy_score(test_labels, predicted_labels))

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/drive/My Drive/BSAN6070/Data/train-mails'